<a href="https://colab.research.google.com/github/cosmicoxytocin/ComfyUI-Colab/blob/main/ComfyColab_DEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # Connect Google Drive
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
Shared_Drive = "" #@param {type:"string"}
#@markdown - Leave empty if you're not using a shared drive

print("\033[0;33mConnecting...")
drive.mount('/content/drive')

if Shared_Drive!="" and os.path.exists("/content/drive/Shareddrives"):
  mainpth="Shareddrives/"+Shared_Drive
else:
  mainpth="MyDrive"

clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

In [ ]:
#@markdown # Install/Update Comfy
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time
import base64
import requests
from urllib.request import urlopen, Request
from urllib.parse import urlparse, parse_qs, unquote
from tqdm import tqdm
import six

if not os.path.exists("/content/drive"):
  print('\033[1;31mGdrive not connected, using temporary colab storage...')
  time.sleep(4)
  mainpth ="MyDrive"
  !mkdir -p /content/drive/$mainpth
  Shared_Drive=""

if Shared_Drive!="" and not os.path.exists("/content/drive/Shareddrives"):
  print('\033[1;31mShared drive not detected, using default MyDrive')
  mainpth="MyDrive"

with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  figitclone = "git clone --depth 1"
  %mkdir -p /content/drive/$mainpth/Comfy
  %cd /content/drive/$mainpth/Comfy
  !git clone -q --branch master https://github.com/comfyanonymous/ComfyUI

with capture.capture_output() as cap:
  %cd /content/drive/$mainpth/Comfy/ComfyUI/
  !git reset --hard
  !git checkout master
  time.sleep(1)
  !git pull
clear_output()
inf('\u2714 Done', 'success', '50px')

#@markdown ---

In [ ]:
#@markdown # ❌ ~~Download Model~~ ❌

#@markdown <h2><font color="red"><b>NOT FINISHED! DO NOT USE</b></font></h3>

#@markdown ## See [Install Guide](https://github.com/cosmicoxytocin/ComfyUI-Colab/blob/main/docs/install_models_sd3.md)
import gdown
from gdown.download import get_url_from_gdrive_confirmation

Use_Temp_Storage = False #@param {type:"boolean"}

#@markdown ---

Model_Version = "SD3" #@param ["SD3", "SDXL", "1.5"]

#@markdown Or
PATH_to_MODEL = "" #@param {type:"string"}

#@markdown Or
MODEL_URL = "" #@param {type:"string"}

def getsrc(url):
  parsed_url = urlparse(url)
  if parsed_url.netloc == 'civitai.com':
    src='civitai'
  elif parsed_url.netloc == 'drive.google.com':
    src='gdrive'
  elif parsed_url.netloc == 'huggingface.co':
    src='huggingface'
  else:
    src='others'
  return src

src=getsrc(MODEL_URL)



#@markdown ---

In [ ]:
#@markdown # Requirements
import subprocess
from tqdm import tqdm
print("\033[1;32mInstalling Requirements...")

with capture.capture_output() as cap:
  %cd /content/
  !apt -y update -qq

  if not os.path.exists('drive/'+mainpth+'/Comfy/libtcmalloc/libtcmalloc_minimal.so.4'):
    %env CXXFLAGS=-std=c++14
    !wget -q https://github.com/gperftools/gperftools/releases/download/gperftools-2.5/gperftools-2.5.tar.gz && tar zxf gperftools-2.5.tar.gz && mv gperftools-2.5 gperftools
    !wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/Patch
    %cd /content/gperftools
    !patch -p1 < /content/Patch
    !./configure --enable-minimal --enable-libunwind --enable-frame-pointers --enable-dynamic-sized-delete-support --enable-sized-delete --enable-emergency-malloc; make -j4
    !mkdir -p /content/drive/$mainpth/Comfy/libtcmalloc && cp .libs/libtcmalloc*.so* /content/drive/$mainpth/Comfy/libtcmalloc
    %env LD_PRELOAD=/content/drive/$mainpth/Comfy/libtcmalloc/libtcmalloc_minimal.so.4
    %cd /content
  else:
      %env LD_PRELOAD=/content/drive/$mainpth/Comfy/libtcmalloc/libtcmalloc_minimal.so.4

torch_packages = [
    "torch",
    "torchvision",
    "torchaudio",
]
extra_index = "https://download.pytorch.org/whl/cu121"
final_packages = [
    "triton==2.0.0",
    "mediapipe==0.9.1.0",
    "addict",
    "yapf",
    "fvcore",
    "omegaconf",
]
for package in tqdm(['jax', 'jaxlib'], desc='Downgrading JAX', unit='package'):
  subprocess.run(f'pip uninstall -y -q {package}', shell=True)

for package in tqdm(['jax==0.4.23', 'jaxlib==0.4.23'], desc='Installing JAX', unit='package'):
  subprocess.run(f'pip install -q {package}', shell=True)

"""for package in tqdm(final_packages, desc="Installing Packages", unit='package'):
  subprocess.run(f'pip install -q -U {package}', shell=True)
"""
for package in tqdm(['xformers'], desc="Installing Xformers", unit="package"):
  subprocess.run(
      f'pip install -q -U xformers --index-url https://download.pytorch.org/whl/cu121',
      shell=True
)

%cd /content/drive/$mainpth/Comfy/ComfyUI
!pip install -qq -r requirements.txt

#clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

In [ ]:
#@markdown # Launch ComfyUI
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)
%cd /content/drive/$mainpth/Comfy/ComfyUI
#model_dir = "/content/drive/"+mainpth+"/Comfy/ComfyUI/models/checkpoints/"

!python main.py --dont-print-server


#@markdown ---